In [4]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.datasets.data_utils import get_file
from simplejson import Decimal
import numpy as np
import random
import sys
import re

path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
text = text[0:20000]
# Get rid of newline characters, join by spaces, and then strip extra spaces
text = re.sub(' +',' ',' '.join(text.splitlines()))
print('corpus length:', len(text))

chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))




Using Theano backend.
Couldn't import dot_parser, loading of dot files will not be possible.
corpus length: 19981
total chars: 49


In [2]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


nb sequences: 6654
Vectorization...


In [41]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
# model.add(Embedding(len(chars), 40))
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


In [42]:

def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    if iteration % 10 == 0:
        json_string = model.to_json()
        open('trump_rnn_lstm.json', 'w').write(json_string)
        model.save_weights('trump_rnn_lstm.h5')
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for iteration in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
6654/6654 [==============================] - 586s - loss: 3.0902    

----- diversity: 0.2
----- Generating with seed: "y. it seems, however"
y. it seems, however        s                           t ete  te                      t                    e t   e  l e           e  t     tn      e         e  e                t    t    e     e          n             e e    e  tt  e    e    e               e      e         e                  e         et          t     e    e        t             t        o   e  t  n  n      o   e        o     t   t  e             

----- diversity: 0.5
----- Generating with seed: "y. it seems, however"
y. it seems, howeveret lee   tns   htnle tea  sotoee i eneee   ost   , mh i t ee neie e   t ititete  ne eete e    l er e i  n  aen n   to n,  e os eti    t e  e  lenie f teeeee en  rear lot  o ia loed e t p nl ot n iy  ohste  sn   et t  l n nc na  p ose eo e ae aut  e f  eensn e o tttat  t

KeyboardInterrupt: 

In [43]:
json_string = model.to_json()
open('trump_rnn_lstm.json', 'w').write(json_string)
model.save_weights('trump_rnn_lstm.h5')

### Loading Keras model and making prediction in javascript

In [3]:
from keras.models import model_from_json
model =  model_from_json(open('trump_rnn_lstm.json').read())
model.load_weights('trump_rnn_lstm.h5')

In [4]:
import json
json_string = open('trump_rnn_lstm.json').read()
mjson = json.loads(json_string)

In [36]:
import pickle
pickle.dump( chars, open( "chars.pkl", "wb" ) )

In [9]:
keras_rnn_layers = mjson['layers']

In [12]:
keras_rnn_layers[2]

{u'activation': u'tanh',
 u'forget_bias_init': u'one',
 u'go_backwards': False,
 u'init': u'glorot_uniform',
 u'inner_activation': u'hard_sigmoid',
 u'inner_init': u'orthogonal',
 u'input_dim': 512,
 u'input_length': None,
 u'name': u'LSTM',
 u'output_dim': 512,
 u'return_sequences': False,
 u'stateful': False}

In [24]:
# Get parameters for Karpathy's LSTM by looping over json object
print(mjson['layers'][5])
mlayer1 = model.layers[5]

# Matrices in an LSTM layer in Keras
# mlayer1.U_c
# mlayer1.U_f
# mlayer1.U_i
# mlayer1.U_o
# mlayer1.W_c
# mlayer1.W_f
# mlayer1.W_i
# mlayer1.W_o
# mlayer1.activation - tanh
# mlayer1.b_c
# mlayer1.b_f
# mlayer1.b_i
# mlayer1.b_o


{u'beta': 0.1, u'target': 0, u'activation': u'softmax', u'name': u'Activation'}


In [30]:
def get_weights(mat):
    # n rows, d columns
    # unravel matrices in row-major order
    mat = mat.T
    if len(mat.shape) == 1:
        d = 1
    else:
        d = mat.shape[1]
    return {
            'n': mat.shape[0],
            'd': d,
            'w': dict(enumerate([Decimal(str(m)) for m in np.ravel(mat)]))
        }

def keras_to_recurrentjs(keras_rnn_model_obj, 
                         keras_rnn_model_json, 
                         vocab, indices_char,
                         char_indices,
                         ltype='lstm',
                        ):
    """
        Convert a Keras RNN object to ReccurentJS object, under the
        assumption that we trained with cross entropy loss, if you 
        want to keep training in javascript
        
        Word embeddings not supported yet
    """
    keras_rnn_json_layers = keras_rnn_model_json['layers']
    keras_rnn_matrix_layers = keras_rnn_model_obj.layers
    assert len(keras_rnn_json_layers) ==  len(keras_rnn_matrix_layers)
    assert len(indices_char) == len(vocab)
    assert len(char_indices) == len(vocab)
    
    num_hidden_layers = 0
    hidden_sizes = []
    letter_size = 0
    recurrentjs_json = {}
    layers = []
    
    # Get number of hidden layers and check that last two layers 
    # are dense and softmax respectively
    for idx, krjl in enumerate(keras_rnn_json_layers):
        if idx == len(keras_rnn_json_layers) - 2:
            assert krjl['name'] == 'Dense'
        if idx == len(keras_rnn_json_layers) - 1:
            assert krjl['name'] == 'Activation'
            assert krjl['activation'] == 'softmax'
        if krjl['name'].lower() == ltype:
            if num_hidden_layers == 0:
                letter_size, vocab_length = krjl['input_shape']
            layers.append(keras_rnn_matrix_layers[idx])
            num_hidden_layers += 1
    
    assert num_hidden_layers > 0, \
        'Found 0 hidden layers with layer type: %s. Only lstm is currently supported' %ltype
    assert (len(keras_rnn_json_layers) - 2) % num_hidden_layers == 0, \
        'Mismatch in number of hidden layers found and number of layers in Keras json'
    assert letter_size > 0, 'Inpute shape of Keras RNN has 0 input shape'
    
    print('>> Converting an %s RNN with %d layers...' % (ltype, num_hidden_layers))
        
    # Construct RecurrentJS model json
    print('>> Creating JSON import to recurrentJS...'
    recurrentjs_json['vocab'] = list(vocab)
    recurrentjs_json['generator'] = ltype
    recurrentjs_json['letter_size'] = letter_size
    recurrentjs_json['indexToLetter'] = indices_char
    recurrentjs_json['letterToIndex'] = char_indices
    recurrentjs_json['hidden_sizes'] = []
    model = {}
    print('>> Loading Keras %s to recurrentJS json...' %ltype
    if ltype == 'lstm':
        for idx, layer in enumerate(layers):
            model['Wox' + str(idx)] = get_weights(layer.W_o.get_value())
            model['Wix' + str(idx)] = get_weights(layer.W_i.get_value())
            model['Wfx' + str(idx)] = get_weights(layer.W_f.get_value())
            model['Wcx' + str(idx)] = get_weights(layer.W_c.get_value())
            
            model['Woh' + str(idx)] = get_weights(layer.U_o.get_value())
            model['Wih' + str(idx)] = get_weights(layer.U_i.get_value())
            recurrentjs_json['hidden_sizes'].append(model['Wih' + str(idx)]['n'])
            model['Wfh' + str(idx)] = get_weights(layer.U_f.get_value())
            model['Wch' + str(idx)] = get_weights(layer.U_c.get_value())
            
            model['bo' + str(idx)] = get_weights(layer.b_o.get_value())
            model['bi' + str(idx)] = get_weights(layer.b_i.get_value())
            model['bf' + str(idx)] = get_weights(layer.b_f.get_value())
            model['bc' + str(idx)] = get_weights(layer.b_c.get_value())
            
    else:
        raise NotImplementedError()
        
    model['Whd'] = get_weights(keras_rnn_matrix_layers[-2].W.get_value())
    model['bd'] = get_weights(keras_rnn_matrix_layers[-2].b.get_value())
    
    # Word embeddings, identity matrix if there are no embeddings
    model['Wil'] = get_weights(np.eye(len(vocab)))
    
    recurrentjs_json['model'] = model
    
    print('>> Done'
    return recurrentjs_json
    
    

In [31]:
recurrentjs_json = keras_to_recurrentjs(model, mjson, chars, indices_char, char_indices, ltype='lstm')

>> Converting an lstm RNN with 2 layers...


In [40]:
mjson['layers']

[{u'activation': u'tanh',
  u'forget_bias_init': u'one',
  u'go_backwards': False,
  u'init': u'glorot_uniform',
  u'inner_activation': u'hard_sigmoid',
  u'inner_init': u'orthogonal',
  u'input_dim': 49,
  u'input_length': None,
  u'input_shape': [20, 49],
  u'name': u'LSTM',
  u'output_dim': 512,
  u'return_sequences': True,
  u'stateful': False},
 {u'name': u'Dropout', u'p': 0.2},
 {u'activation': u'tanh',
  u'forget_bias_init': u'one',
  u'go_backwards': False,
  u'init': u'glorot_uniform',
  u'inner_activation': u'hard_sigmoid',
  u'inner_init': u'orthogonal',
  u'input_dim': 512,
  u'input_length': None,
  u'name': u'LSTM',
  u'output_dim': 512,
  u'return_sequences': False,
  u'stateful': False},
 {u'name': u'Dropout', u'p': 0.2},
 {u'W_constraint': None,
  u'W_regularizer': None,
  u'activation': u'linear',
  u'activity_regularizer': None,
  u'b_constraint': None,
  u'b_regularizer': None,
  u'init': u'glorot_uniform',
  u'input_dim': None,
  u'name': u'Dense',
  u'output_dim':

In [32]:
import simplejson as json
with open('recurrentjs_json.json', 'w') as outfile:
    json.dump(recurrentjs_json, outfile)

In [19]:
# 2 layers, hidden_sizes: 10, letter_size: 5, distinct_chars: 51
karpathy_js = {"hidden_sizes":[10,10],"generator":"lstm","letter_size":5,"model":{"Wil":{"n":51,"d":5,"w":{"0":-0.17132319367495344,"1":-0.11624874039230527,"2":-0.4157855432343936,"3":-0.11159957084009939,"4":0.38240035176085163,"5":0.24955151229384553,"6":0.30476162147478203,"7":0.2487588994894359,"8":-0.3501258562691828,"9":-0.3184433097063579,"10":0.343031945723342,"11":0.43795780806167717,"12":0.14706821630099728,"13":-0.22296714526674588,"14":-0.4235127913308575,"15":-0.045058257607213534,"16":-0.020792894031497925,"17":0.01956414616198482,"18":0.01696580820213418,"19":0.07232469817836613,"20":-0.003640541292856307,"21":0.046224079164777214,"22":0.06430204593950092,"23":-0.04457930350477064,"24":-0.02312523207868217,"25":0.06197519824670983,"26":0.06665514261861048,"27":-0.006360242440434909,"28":-0.03200424355936682,"29":-0.022991084897739485,"30":-0.38967336870527347,"31":-0.31546888893613584,"32":0.3389257522396783,"33":-0.27030416403963226,"34":0.3283379051529053,"35":-0.021564736646081335,"36":0.04543672267164039,"37":-0.03460185309780547,"38":0.035515700263421726,"39":-0.04959325379199793,"40":-0.04515616699634146,"41":-0.04283854698056704,"42":-0.028011808900619262,"43":-0.01867208570306777,"44":0.03637597016042207,"45":-0.02666229800689297,"46":-0.018636498886203088,"47":0.057024921281484904,"48":-0.03771636431294868,"49":0.031027055990954434,"50":0.31956756422777494,"51":0.2529353283261417,"52":-0.2742991195442641,"53":0.3314609662427911,"54":-0.2469024165117735,"55":-0.657175332294864,"56":-0.7682041772436841,"57":-0.36362404784770436,"58":0.47553070425132493,"59":1.0069096682817367,"60":-0.5874151750836173,"61":-0.6442655785557951,"62":-0.6935945736627201,"63":1.0902093639209642,"64":0.4856241614968972,"65":-0.6526720520336189,"66":-0.573911832617066,"67":-0.78093182317496,"68":0.6059269783151607,"69":-0.31817384346592065,"70":0.2937687091977239,"71":0.5975919331074165,"72":0.17251753383202328,"73":0.38244738182196325,"74":-0.5090214898985937,"75":-0.08813034828556468,"76":-0.12469859394036178,"77":0.09904971307943386,"78":-0.22509979579040512,"79":0.03176090064045382,"80":0.07217064331586273,"81":0.3620250066614029,"82":-0.25272965832654615,"83":0.385255703741435,"84":-0.4516904352761424,"85":0.4777905277194139,"86":-0.323705318464516,"87":1.873901191977973,"88":-0.6849524326124243,"89":0.5023618413481001,"90":-0.21575645621050316,"91":-0.3581504029932653,"92":-0.09481207916323398,"93":-0.07224897524546628,"94":0.34768386984553445,"95":0.4247035852941643,"96":0.5294261890909987,"97":0.10213293660983128,"98":0.14212676701164517,"99":-0.24507253009044697,"100":0.047104755286832764,"101":0.42520620308726537,"102":0.23594880098626492,"103":-0.645226130219896,"104":-0.4728406530944769,"105":-0.15550404465571577,"106":-1.260427401241417,"107":1.6304851408259295,"108":-0.9383228755999006,"109":1.2627321339752677,"110":-0.9055771347511501,"111":-0.6609944418604181,"112":-1.7133592077799922,"113":1.8991003844945464,"114":0.3003028897363527,"115":-0.2819269417804163,"116":-0.8934207629809784,"117":-0.2407070506524277,"118":0.9236881538829953,"119":1.0175905710363504,"120":-0.6997048747574399,"121":-0.6821749460114802,"122":-0.7577811283021517,"123":1.381143683430282,"124":0.677129249473886,"125":-0.9445782289469189,"126":-0.9879211562128538,"127":-0.9445033496996196,"128":0.2299224399103816,"129":0.4352121047459176,"130":-0.14757193693171794,"131":0.3003967914986674,"132":0.058824222942449315,"133":-0.4369494684361104,"134":-0.3167102374091063,"135":-0.32104547034932207,"136":-0.5514169080382895,"137":0.2945162644370919,"138":-0.07015264748957172,"139":0.5704312266681644,"140":0.2647968642358632,"141":0.7395369201695827,"142":-0.3456880950289909,"143":-0.028110907981160678,"144":-0.058946926080591135,"145":0.10195912496387827,"146":0.08907697998704951,"147":0.1493739164800201,"148":-0.3801862710155069,"149":-0.05985334769836742,"150":0.8852670873015899,"151":-0.7242890190717463,"152":0.20252800518812836,"153":-1.0063869598555135,"154":0.7867127936112815,"155":-0.21583952607031187,"156":-0.2104671020673075,"157":-0.7400870071253196,"158":0.36994651917463156,"159":0.8191316179660305,"160":-0.05720295121465211,"161":0.012852970408672336,"162":-0.04005572897815662,"163":-0.07089721948402784,"164":-0.011855466238020307,"165":0.21029584813115357,"166":-0.7982607203889073,"167":1.7063102199926192,"168":-1.0613463003684913,"169":1.018052256320191,"170":-0.717703279322699,"171":-1.0584636881015677,"172":-0.5642832939483141,"173":0.5665728023555766,"174":0.28448661826172855,"175":-1.2720583278252693,"176":-1.0550969237277816,"177":-1.324176275343273,"178":1.0961046255148463,"179":0.6142188665168663,"180":-0.02244645444479254,"181":-0.024447782067218843,"182":0.05000717320594053,"183":-0.03266221972177784,"184":-0.008474054362039386,"185":0.0606011123873979,"186":-0.07175739066379683,"187":0.041302711969147594,"188":-0.011824921673822092,"189":0.007493377269880154,"190":-0.42214981528142564,"191":-0.5432153480847063,"192":0.052595932248328564,"193":-0.14661648201593655,"194":0.6465096813748216,"195":-0.07127426119498582,"196":-0.06744055713892191,"197":0.06982635507434627,"198":0.0181353287129823,"199":0.05104224338081625,"200":-0.07707598504633945,"201":-0.03775396723369176,"202":-0.026144201665430168,"203":0.056919630935350476,"204":-0.056598341997276584,"205":0.07149543510296923,"206":0.049808283315991644,"207":-0.05176628126345144,"208":-0.017738038794062678,"209":-0.07104908730204008,"210":0.3012334336491717,"211":0.3256433004383692,"212":0.30732795787757844,"213":-0.31198239551460066,"214":-0.2606409464888686,"215":0.0016575229974644294,"216":0.041801007235829094,"217":0.02815680208789231,"218":-0.06049759880242057,"219":0.005952392199127274,"220":0.35862064130608073,"221":0.33945484659171976,"222":0.3551434716775752,"223":-0.24509031992970193,"224":-0.28640753851321005,"225":0.04775759800918677,"226":0.07987376623415947,"227":-0.04979787866881487,"228":-0.00649244980915575,"229":-0.04104398598910824,"230":0.016262821005422024,"231":0.018308801076463054,"232":-0.03837322158073303,"233":0.019384519654546552,"234":-0.001223313504553396,"235":0.24625514167533977,"236":0.24210553396610246,"237":0.27164780694132146,"238":-0.33341391563907824,"239":-0.28514070637368505,"240":0.01749914048657941,"241":0.0715397989247506,"242":0.06252901130201305,"243":0.004914035559986979,"244":0.04454452959636728,"245":0.02189041265241983,"246":0.06462234297428454,"247":-0.0008259405886558713,"248":0.07749646327038447,"249":-0.05723532737463553,"250":-0.05168262083125143,"251":0.053460146320478005,"252":-0.0379186802496021,"253":-0.05947315816375015,"254":0.07556463220630819}},"Wix0":{"n":10,"d":5,"w":{"0":0.12944173518806515,"1":-0.14967301016979145,"2":0.49855507937203813,"3":0.48346087493232803,"4":0.9183530674722862,"5":0.1889700569140979,"6":0.2435945903404655,"7":0.21050832125555402,"8":-0.39647029817693924,"9":0.4653472166989592,"10":0.5957617106804759,"11":0.5292127394130034,"12":1.0020040183680474,"13":-0.531979543777705,"14":-0.07512687117920458,"15":0.4222613899355975,"16":-0.05030226655863343,"17":1.1206304012821835,"18":-0.5259083603478024,"19":0.5211650491264775,"20":0.6053495411326988,"21":1.4320918505241034,"22":1.1026042708781367,"23":0.08495928641440925,"24":-0.15214561657079317,"25":-0.13469129540544725,"26":-0.23765522933784772,"27":-0.25468212167075854,"28":-0.2575312525452722,"29":0.28627728127301655,"30":0.433886309916972,"31":0.07448705355697141,"32":1.0365572441124693,"33":-0.15502657767315325,"34":0.9031168461205821,"35":0.20600153843965627,"36":-0.1473112676138849,"37":0.36938358236842356,"38":-0.7335987297277087,"39":0.5722119822782755,"40":0.02634060633445899,"41":0.26134224791892174,"42":-0.24408503259821432,"43":0.7290561613753919,"44":-0.23292249521452335,"45":-0.7502245679437755,"46":-0.7701905391254784,"47":-0.7505013631764075,"48":0.9854058398379573,"49":0.35360940184702333}},"Wih0":{"n":10,"d":10,"w":{"0":0.760877402772288,"1":-0.1839243727258986,"2":-0.3473169448888565,"3":-0.7396710191537202,"4":-0.18532698632395125,"5":-0.07796861100672464,"6":0.3674494023068939,"7":0.5914004410470333,"8":1.5419831630063439,"9":0.7533486284548149,"10":-0.5008352075224076,"11":-0.41297021438441706,"12":-0.31442394647746474,"13":-0.2554113445361892,"14":-0.3252539938883976,"15":-0.2889162073693918,"16":-0.5645647445769953,"17":0.4244684122862551,"18":0.881645711834604,"19":-0.4066288322059843,"20":-0.18781719064693453,"21":-0.14587703566520938,"22":0.3557716666678953,"23":0.2971997272120416,"24":0.013565846281159727,"25":-0.027983884386475418,"26":-0.08983260551735057,"27":-0.1537909057353419,"28":1.1072706359636197,"29":0.405021230926649,"30":0.2159451587780776,"31":-0.20597454622454714,"32":0.026693978024883713,"33":0.09524556951164609,"34":-0.20501719153627987,"35":-0.14676520897424583,"36":-0.3003719034880019,"37":-0.1836810025417851,"38":1.290008423889415,"39":0.5675129275916473,"40":-0.13208636555457848,"41":0.04106439812935151,"42":-0.38301365612344623,"43":0.05280289768290876,"44":-0.40820783585628545,"45":0.30744050164321435,"46":0.024242478762364246,"47":0.15818093062770117,"48":1.168357828824448,"49":0.36443326963182077,"50":-0.5312866234432047,"51":-0.19117133539566908,"52":-0.15198834137402806,"53":-0.07604554173184935,"54":-0.2767233646404208,"55":-0.06346035392793989,"56":-0.3768655381625197,"57":0.04390523920288474,"58":0.5666773156183147,"59":-0.3975311790705253,"60":0.13407307747157096,"61":-0.23080242300115628,"62":-0.8782700264473006,"63":-0.911117008328518,"64":-0.43545551598760884,"65":-0.10423682004759452,"66":0.012697441098916878,"67":0.5495943053933864,"68":1.3569106038224228,"69":0.24415531406153856,"70":-0.3566717887396847,"71":-0.25430824061620616,"72":0.014572831352248058,"73":0.013292467746564611,"74":-0.20302002567464214,"75":-0.12296021663756611,"76":-0.409226443917198,"77":0.04330094215763469,"78":0.7953479554190274,"79":0.03427192478290389,"80":1.1631609143714332,"81":0.74887092051587,"82":0.8127560457918181,"83":0.5059051729893473,"84":0.8858605647852458,"85":0.6081485756460868,"86":1.2383010410569546,"87":-0.640663584575968,"88":-0.5817482827141677,"89":1.057354726415856,"90":0.09498267377105854,"91":-0.13884981429676022,"92":-0.08954772364194134,"93":-0.29015256929262956,"94":-0.2587669204063209,"95":-0.00894887257568474,"96":-0.07000041549462563,"97":0.16237277465519057,"98":0.7640090310779941,"99":0.40878963887982644}},"bi0":{"n":10,"d":1,"w":{"0":-0.06603703105456249,"1":-0.6028944060542918,"2":-0.12580235971974668,"3":-0.007781810450068274,"4":-0.6351582432307036,"5":-0.49255169573682794,"6":-0.6235313563854962,"7":-0.4183585216003331,"8":0.8149819227638915,"9":-0.18278467854897304}},"Wfx0":{"n":10,"d":5,"w":{"0":-0.76464678058605,"1":-1.3806559522699453,"2":-0.5539049482395051,"3":0.570727432132127,"4":1.0064417075809455,"5":0.0137057416669681,"6":0.6017164191202065,"7":0.1747815561900603,"8":0.23482823719256982,"9":-0.5977231455079409,"10":-0.06904082579912908,"11":0.6524539149282943,"12":-0.9001607744286165,"13":0.32251094729508695,"14":-0.7670286670470531,"15":0.360955077026742,"16":0.3745257908606461,"17":-0.2992601773424399,"18":-0.33463651002385875,"19":0.22459960363076206,"20":0.3035776363111975,"21":0.5778572849983659,"22":0.6081583608785063,"23":-0.31033528863463233,"24":-0.2941737956841991,"25":0.09413412633857399,"26":0.23839743679663622,"27":-0.061916532976895984,"28":0.02444065584731942,"29":0.3513909088387114,"30":0.16657584635001005,"31":-0.5391114028462377,"32":0.6905383174986877,"33":-0.07513101233911774,"34":0.6045585752334883,"35":0.09195894130409518,"36":0.7724871349418359,"37":-0.6442529369200262,"38":0.4203485295653322,"39":0.09073797348246987,"40":1.0491395664003196,"41":0.9717168426179095,"42":0.9923916294183592,"43":-0.6618945430157704,"44":-0.08891282563904992,"45":-0.34005623498642806,"46":-1.1320788309015168,"47":0.12600037034509526,"48":-0.17412220879364193,"49":1.00978275987806}},"Wfh0":{"n":10,"d":10,"w":{"0":-0.4704883652522163,"1":-0.3141297247243428,"2":-0.26340990567873607,"3":-0.20798420821611713,"4":-0.37096603683849216,"5":-0.39088299571536594,"6":-0.5156273311608062,"7":0.25523329319311505,"8":0.493903806489037,"9":-0.39945518369722777,"10":-0.32720479839975103,"11":-0.3393415198955775,"12":-0.25845044415380275,"13":-0.23431791243878813,"14":-0.28147076002524274,"15":-0.1430193721043562,"16":-0.32159997240607685,"17":0.2732188556305262,"18":1.007264866710023,"19":-0.10434875072465406,"20":-0.38255752659002656,"21":-0.12135505035427863,"22":-0.049918479620346165,"23":-0.1589396071946621,"24":-0.12472940914972322,"25":-0.2569558764623796,"26":-0.331566318379367,"27":0.21355694808649492,"28":0.7339704146120968,"29":-0.2644853300918797,"30":-0.40486479374978473,"31":-0.2262079195463167,"32":-0.35851840923777417,"33":-0.3134494804469547,"34":-0.19363810151569538,"35":-0.12390824970275807,"36":-0.31140854436949383,"37":0.24682614171068087,"38":0.7158213685791885,"39":-0.36364124195973707,"40":-0.3614152418949927,"41":-0.3330751848385908,"42":-0.5308666145376277,"43":-0.37899819273070146,"44":-0.2604182788595257,"45":-0.2790867743748781,"46":-0.2114955122440125,"47":0.48761964101794386,"48":1.164537127052841,"49":-0.23929408169742278,"50":-0.21754319949738712,"51":-0.22683666735356997,"52":-0.14463675601736506,"53":-0.14033053638660836,"54":-0.2034894093922177,"55":-0.03176913520651846,"56":-0.24510232187733355,"57":0.07571049865923948,"58":0.73391442432857,"59":0.025023990334155328,"60":-0.818152753644528,"61":-0.3895070438170005,"62":-0.47697257101180773,"63":-0.09568164914801626,"64":-0.3542819707570441,"65":-0.5405836308535377,"66":-0.5570923379441234,"67":0.47674278254806157,"68":0.26684832905321637,"69":-0.87308015209338,"70":-0.25713493547658856,"71":-0.1312898640781697,"72":-0.2531118703624971,"73":-0.34691416431277694,"74":-0.1701205096643044,"75":-0.07378665354372456,"76":-0.1445197942382925,"77":0.23747336583365095,"78":0.5736641251816671,"79":-0.15199284528132892,"80":0.2981385642657732,"81":0.592951695337501,"82":0.52153756184142,"83":0.6184263028240211,"84":0.5079986696365265,"85":0.45475399514138926,"86":0.28844417660145105,"87":-0.4829295528096664,"88":-0.28208310280749416,"89":0.34754985918774484,"90":-0.37281195747306395,"91":-0.22351038482965632,"92":-0.22669505517450445,"93":-0.16727337455303923,"94":-0.31769363315231225,"95":-0.1077888714781224,"96":-0.3500227890379077,"97":0.2429605322925884,"98":0.5741374411436698,"99":-0.16492946482160015}},"bf0":{"n":10,"d":1,"w":{"0":-0.43011606165804994,"1":-0.20890130008982677,"2":-0.008356552543163559,"3":-0.2974408509703736,"4":-0.011340308606136884,"5":-0.35538972060889207,"6":-0.7334450772370885,"7":-0.21315477073270883,"8":0.11510511933558615,"9":-0.47171908100898}},"Wox0":{"n":10,"d":5,"w":{"0":-0.5402781242485643,"1":-0.9424142550224822,"2":-0.02118424477856045,"3":0.6184689970920897,"4":1.1651392943857053,"5":0.16008445087684228,"6":0.4547768930366068,"7":0.3536172311182345,"8":-0.4360845720447984,"9":-0.2524821257844279,"10":-0.1915018428490653,"11":-0.08659733056931972,"12":-0.00772717147280734,"13":0.13258231543536597,"14":-0.5068191276627899,"15":0.0907489331559838,"16":-0.46881161845724123,"17":0.40848626732601634,"18":-0.5613819541731438,"19":1.0896251555615502,"20":-0.27518628725664895,"21":0.5122051556782063,"22":0.34969333965661753,"23":0.12978648715387825,"24":-0.3742503176889097,"25":0.2896897977829446,"26":0.3672517255728755,"27":0.3353964704800585,"28":-0.39747940580541263,"29":0.13867293630267039,"30":-0.13066565936603372,"31":-0.6456163599839161,"32":0.7285375331385292,"33":-0.08348398052786207,"34":0.76964554982182,"35":0.09508469017919828,"36":0.10418756942232321,"37":0.096417103497968,"38":-0.25662908525670847,"39":0.3741525986674566,"40":-0.4057276522743967,"41":0.005395725978075822,"42":-1.2773241459092655,"43":1.5454561753047658,"44":-0.2642338865029656,"45":-0.5395245673878875,"46":-0.8802193852053863,"47":-0.24512382046360048,"48":0.44224854067654185,"49":0.6996461019327058}},"Woh0":{"n":10,"d":10,"w":{"0":0.05775164987750646,"1":-0.11116233844956085,"2":-0.23216105223183284,"3":-0.31909542870453067,"4":-0.10680352816391817,"5":-0.14911468846628062,"6":-0.08505477620643145,"7":0.3252283303187904,"8":0.9506814181121817,"9":0.14935412363501516,"10":-0.3307327827586344,"11":-0.3509057037653612,"12":-0.3291219357019492,"13":-0.1637188798691807,"14":-0.3643525867745723,"15":-0.33508265423016775,"16":-0.27600086814652225,"17":0.28864050358187165,"18":0.7494970638909758,"19":-0.29634148093167506,"20":-0.13896207116585074,"21":-0.09420054401661562,"22":-0.04576278979287873,"23":-0.06672770481436952,"24":-0.06559921405932116,"25":-0.14915267703906868,"26":-0.15371453778967079,"27":0.09017027161132452,"28":1.026126625100244,"29":0.12598654803052964,"30":-0.1738779900017154,"31":-0.1625273998404782,"32":-0.07092359816981467,"33":-0.15575029773649973,"34":-0.0818996402845015,"35":-0.1595398738415219,"36":-0.2930994762235511,"37":0.06987522409979655,"38":1.259226292891744,"39":0.19092160774870232,"40":-0.10381068687804976,"41":-0.10747748594797354,"42":-0.21100064913722968,"43":-0.14691075710583124,"44":-0.22726002945140028,"45":-0.14747061761420005,"46":-0.09853084418207116,"47":0.1664822813943223,"48":1.2649803951251262,"49":-0.0031970139359980346,"50":-0.4458590122031589,"51":-0.26514619753657626,"52":-0.2983702915380871,"53":-0.2228311961854501,"54":-0.2250068075750684,"55":-0.2513797290701451,"56":-0.367537205429009,"57":0.12001099995056094,"58":0.5354413189240002,"59":-0.3243693293331232,"60":-0.18179085380649213,"61":-0.15361183895224018,"62":-0.2035945421004832,"63":-0.33722586338505817,"64":-0.1504676962441146,"65":-0.16258527221164187,"66":-0.07542896004058948,"67":0.3422429520404517,"68":1.1998768519464256,"69":-0.17949770730633757,"70":-0.30800726623430946,"71":-0.27880568841061665,"72":-0.21290234620660384,"73":-0.30672425795380526,"74":-0.2200586750355758,"75":-0.22304757816601434,"76":-0.3017463377261718,"77":0.14162251382852492,"78":0.9135773012222022,"79":-0.09443590110546268,"80":1.1731024253494042,"81":0.6092294438419752,"82":0.8292596658799494,"83":0.47150645921871776,"84":1.1088624896395265,"85":0.4752854283480703,"86":1.0999123936871968,"87":-0.6405701357664788,"88":-0.3187103040479112,"89":0.8937870505117365,"90":0.1581336070172322,"91":-0.2846182517260923,"92":-0.3624822124750048,"93":-0.30164033467837437,"94":-0.3240753023613224,"95":-0.10821102865930506,"96":-0.043543319279164695,"97":0.30030351828098,"98":1.0397711946530959,"99":0.347377914480026}},"bo0":{"n":10,"d":1,"w":{"0":0.1829743361785246,"1":-0.297794530149536,"2":0.07931779110946899,"3":-0.029868739735111352,"4":-0.18213186916801755,"5":-0.4554960493738334,"6":-0.20735442180051405,"7":-0.272465541809876,"8":0.49128766507380456,"9":-0.14097134092324276}},"Wcx0":{"n":10,"d":5,"w":{"0":-0.6198448256477012,"1":-1.737199320820877,"2":0.0914706483818429,"3":-0.06905994098640834,"4":1.0471658306018559,"5":-0.5944512130948901,"6":0.1638909018289337,"7":0.1430375982510197,"8":0.05804258772292262,"9":-0.048933775341621376,"10":-0.08002847911883938,"11":0.9041847651177848,"12":-0.30825837450160587,"13":-0.07909679262070693,"14":-1.3632255443457935,"15":0.1580930332029714,"16":1.2798790397867863,"17":-0.3960773006452961,"18":0.38022971737077566,"19":-1.506739163979379,"20":-0.21818255155711586,"21":0.1846159728018691,"22":0.19420793199958047,"23":-0.33931028884290904,"24":0.024091247350412195,"25":-0.2997775444923841,"26":-0.09368345458395451,"27":0.25215607958150016,"28":-0.1730891621798931,"29":-0.0497454980149338,"30":-0.36478217624569836,"31":-0.9512862098934606,"32":0.5287952780718082,"33":-0.4143880904363949,"34":0.8078862125299484,"35":0.16934907977790606,"36":-0.7912864714413526,"37":0.225184537579561,"38":-0.11385994294218718,"39":0.7117753218548852,"40":-1.1908559367738993,"41":-1.4135887204437774,"42":-1.1638893423604357,"43":1.2170810899208337,"44":0.3490024906775249,"45":-0.8079334789142528,"46":-1.1086027242128176,"47":-0.41018372597572716,"48":0.5180094445384862,"49":0.36554642630631745}},"Wch0":{"n":10,"d":10,"w":{"0":-0.32573591925367207,"1":0.04287420312649911,"2":0.2504876357220231,"3":-0.006779214740309132,"4":0.22727077119165734,"5":0.2272735425661579,"6":-0.5846232490040563,"7":-0.1482294913441801,"8":-0.20018198330475107,"9":-0.12382306665837145,"10":-0.17648794259908474,"11":-0.45136891693641973,"12":-0.4967177844978571,"13":-0.3599103617411474,"14":-0.35206873160570795,"15":-0.08099697244992415,"16":-0.4789298602264224,"17":0.4925208955424372,"18":0.42206000690661594,"19":0.21447903069885313,"20":-0.5660872875891145,"21":-0.29048074255313777,"22":-0.3270361749582753,"23":-0.2629364412457491,"24":-0.21855740043657526,"25":-0.05944205895090954,"26":-0.37516339443077107,"27":0.13371245661398812,"28":0.1291073796683252,"29":-0.014168753908481962,"30":-0.7552749870188863,"31":-0.3848505264611308,"32":-0.3452158405978272,"33":-0.47266530767072834,"34":-0.460972343911993,"35":-0.3114277456288584,"36":-0.2946260682510217,"37":0.24163748933003842,"38":0.004386142793505993,"39":-0.30661795536017844,"40":0.15616267912417797,"41":0.44774163710231996,"42":0.7541346554630279,"43":0.6506994659612957,"44":0.40235841400099276,"45":0.7625919850923251,"46":-0.09834869359968437,"47":-0.7461889837861151,"48":0.3191628811052146,"49":1.0515831910204825,"50":-0.5261052680545364,"51":-0.6387945365458919,"52":-0.670955432737903,"53":-0.511216871682157,"54":-0.6855217862933936,"55":-0.45125011734416237,"56":-0.6397740827823313,"57":0.5094784904343151,"58":0.44892140912560385,"59":-0.22363839577135605,"60":-0.20469132565043813,"61":0.26849351351790496,"62":0.34444072578989715,"63":0.40154383657579323,"64":0.33258766894488584,"65":0.5104224196050483,"66":-0.473895904936847,"67":-0.5282793228593157,"68":0.07068500014866817,"69":0.629208921355504,"70":0.6484569324445674,"71":0.46277782599168815,"72":0.2569455765515499,"73":0.3281942107245616,"74":0.43157405428263146,"75":0.3325758874678274,"76":0.4096781219621768,"77":-0.2656939896962576,"78":-0.2822368185796438,"79":0.4238411275364045,"80":-0.27717850613536976,"81":-0.7160733103946557,"82":-0.9691800378581578,"83":-0.8107115301606165,"84":-0.9161002378410084,"85":-0.5864512459912434,"86":-0.5218534059217157,"87":0.7080481588660877,"88":0.42972197404889906,"89":-0.2061274505670005,"90":-0.3929354141456519,"91":-0.402013941667643,"92":-0.341463683089152,"93":-0.3000605489246669,"94":-0.44143477031702505,"95":-0.4736439044576892,"96":-0.6845748526013927,"97":0.4955947880597068,"98":0.4539603070759341,"99":-0.34176108195078014}},"bc0":{"n":10,"d":1,"w":{"0":0.2868230952074982,"1":0.5589280540159046,"2":0.5932440617057501,"3":0.5551372168509799,"4":0.29034074873003035,"5":0.3640900745596653,"6":0.0862817655121019,"7":-0.4469663294718669,"8":-0.26870988122647493,"9":0.36827428609782703}},"Wix1":{"n":10,"d":10,"w":{"0":-0.24761647008012072,"1":0.02262089475169002,"2":0.027680441774092478,"3":0.008779337829543078,"4":0.017041959931681526,"5":-0.031458048672338916,"6":-0.19991188276260174,"7":-0.002284458679634236,"8":-0.10583670086135866,"9":-0.024294289172848847,"10":-0.49931943122822514,"11":-0.1424227139943169,"12":0.025137685761134676,"13":0.08056525706814184,"14":-0.062041703924760105,"15":-0.1761868124066061,"16":-0.4366057377456668,"17":0.1021816664153014,"18":0.07420347286001226,"19":-0.21879851747548232,"20":-0.10799262814739574,"21":-0.4664622620593357,"22":-0.776817691219513,"23":-0.8313247134402638,"24":-0.5664185765793118,"25":-0.10301159866842095,"26":-0.2062138452613123,"27":0.3378596575400759,"28":0.1355748536591864,"29":-0.410498417288639,"30":-0.10669659236121554,"31":-0.3441154347319323,"32":-0.4831260398100188,"33":-0.669251840216012,"34":-0.1800402897440069,"35":-0.3653658981048715,"36":0.0841413056515467,"37":0.48468686482051415,"38":-0.4243277139080867,"39":-0.10538281541535455,"40":-0.43681577629198304,"41":0.06847313438144612,"42":0.0075458807855567885,"43":0.14609378892303385,"44":0.04629862676439565,"45":-0.10824849506002383,"46":-0.46181959756607016,"47":0.017982182590850142,"48":-0.09891963786178196,"49":-0.1645931024671163,"50":-0.5656179219923326,"51":-0.15057010106345264,"52":-0.08797527358548299,"53":-0.05941864218884181,"54":-0.061129141000311686,"55":-0.25088940611884847,"56":-0.49517480223852506,"57":0.12618930769680767,"58":0.11934941076620756,"59":-0.13610499101411683,"60":-0.2838648175073083,"61":-0.34931932320044434,"62":-0.22159024682216577,"63":-0.329840198951167,"64":-0.35173260067215945,"65":-0.13528652052288376,"66":-0.23459592398164683,"67":0.1568957135649482,"68":0.9957235531429842,"69":0.1679874327515301,"70":0.0008050675771619376,"71":0.18000085785833347,"72":0.21849766868513415,"73":0.17845705253142277,"74":0.24684935634146993,"75":-0.21576811499333862,"76":-0.21872855775705935,"77":-0.03480832466112084,"78":0.03840428195833406,"79":0.05646579193266649,"80":0.2673592196152658,"81":-0.06266709514414676,"82":-0.04506084830700812,"83":-0.1860241313359354,"84":0.001140345563035966,"85":-0.025184124595394167,"86":-0.18118085511315557,"87":0.062080485707911005,"88":0.22399826163346778,"89":0.1814534581963804,"90":-0.19952061381590902,"91":-0.000687157044067287,"92":0.05404143186038377,"93":0.11681316753680608,"94":0.05340383848914586,"95":-0.03368165536724141,"96":-0.2127808149424682,"97":0.013714623936844942,"98":0.025365360865575528,"99":-0.04939943306413602}},"Wih1":{"n":10,"d":10,"w":{"0":-0.6192508072301838,"1":0.5145077897593704,"2":-0.493010828990388,"3":0.7126871006829832,"4":-0.5509938871547072,"5":-0.4774457569170174,"6":0.005201336691714102,"7":-0.5749696605109224,"8":-0.5762334258358085,"9":0.5817192539815594,"10":-0.38778740751186835,"11":0.37639394661249026,"12":-0.24881526190858935,"13":0.9880369074957968,"14":-0.3690281406642413,"15":-0.33738228188120034,"16":-0.08019676038619873,"17":-0.44354735215895846,"18":-0.33464830910242316,"19":0.39711917831157967,"20":0.3836690817054826,"21":-0.3177919681718846,"22":0.3820683020662648,"23":0.6659800291716618,"24":0.32321484431312947,"25":0.27674801858126435,"26":-0.20913806340656088,"27":0.3960828315048393,"28":0.2904679541344067,"29":-0.2721591879354337,"30":0.25555625574022056,"31":-0.2381203432253074,"32":0.33033724878802156,"33":0.006020758403380781,"34":0.31167757421542314,"35":0.205654792931004,"36":0.050884871366777266,"37":0.3196469478281161,"38":0.31380025493699343,"39":-0.3143636516463218,"40":-0.5799792658951652,"41":0.5106642817646118,"42":-0.5156316307254019,"43":0.7973313854472787,"44":-0.5423950241838402,"45":-0.5489725637543373,"46":-0.004015204156431508,"47":-0.47016235147265145,"48":-0.4848597649352665,"49":0.5569051074740138,"50":-0.402679509313955,"51":0.4283917364583751,"52":-0.44759466153427113,"53":0.9730807533271107,"54":-0.37664210157936234,"55":-0.36246849283486576,"56":-0.10496486876323309,"57":-0.4968266452047432,"58":-0.4194360387070253,"59":0.4796761994575185,"60":0.5938022276652785,"61":-0.4901247803833925,"62":0.5196471192561585,"63":0.2145161066566739,"64":0.5355158087986039,"65":0.5151160909612964,"66":0.04783226034551396,"67":0.5820597648028587,"68":0.5552918079513802,"69":-0.5634557468196864,"70":-0.5681692083045055,"71":0.5950586057381437,"72":-0.2728950145714206,"73":0.9648100048629451,"74":-0.5458838403150627,"75":-0.49465183117990463,"76":-0.0553756921474823,"77":-0.5142597746795936,"78":-0.48157098148914257,"79":0.4998457366977168,"80":-0.28558662781989225,"81":0.2972417079150355,"82":-0.13918609281603847,"83":0.9481016577586235,"84":-0.23065495808947029,"85":-0.21775811210113935,"86":-0.1044722643696335,"87":-0.2611000638331035,"88":-0.275110710937241,"89":0.2857914120467314,"90":-0.6348504000666588,"91":0.5749302898887654,"92":-0.5033229172252376,"93":0.7158562266364877,"94":-0.4913974596999284,"95":-0.5230402064293025,"96":0.039212464323398366,"97":-0.506312253423407,"98":-0.588469631107315,"99":0.5444035183763001}},"bi1":{"n":10,"d":1,"w":{"0":0.5874528540381208,"1":0.4513469742538154,"2":-0.2030525584757064,"3":-0.1634350362945774,"4":0.5227805934743744,"5":0.4337054804306687,"6":-0.46032681207896414,"7":0.4551995870762149,"8":0.45355071283544857,"9":0.5663225165222433}},"Wfx1":{"n":10,"d":10,"w":{"0":-0.14249393546901531,"1":-0.08966736122947687,"2":-0.06622726845850059,"3":0.012269716387079344,"4":-0.06277535751414316,"5":-0.039093121889594545,"6":-0.11402273664069869,"7":0.09713994817551021,"8":0.09209336693251877,"9":0.015232157085328728,"10":-0.12373706674286578,"11":-0.09825684482307405,"12":-0.02776521722644297,"13":-0.1439791233615375,"14":-0.027971457596865226,"15":-0.020331052693424365,"16":-0.15429855534204806,"17":-0.0359556101445117,"18":0.26024335961008704,"19":0.15089454596686017,"20":-0.18882875597097737,"21":-0.6084580093482502,"22":-0.7592692518444356,"23":-0.7444241179135885,"24":-0.38501312328165566,"25":-0.1186523465614158,"26":-0.04269393264954381,"27":0.25650366084631115,"28":0.559003014187225,"29":-0.0228024283818504,"30":-0.33560468133435767,"31":-0.5531932249000111,"32":-0.5370311256006253,"33":-0.5178441603254169,"34":-0.44715432614975814,"35":-0.606751704138093,"36":-0.33465358188306626,"37":0.743906649728067,"38":-0.12249008361451169,"39":-0.09858360100581708,"40":-0.06507249106383747,"41":-0.11033394259445174,"42":-0.1248308115664244,"43":-0.03809195553831664,"44":-0.14687854803634195,"45":-0.09130699534221125,"46":-0.19703655469123157,"47":0.055786165035475614,"48":0.12530923761848092,"49":0.07120229556232686,"50":0.004641015259505761,"51":-0.03979464817044729,"52":0.010709870068160026,"53":-0.0481914007600554,"54":0.00012234129356407386,"55":0.007546354337050896,"56":-0.226324414801218,"57":0.07281786849924486,"58":0.18899678268088604,"59":0.2653597455046078,"60":-0.29827098620624853,"61":-0.16002690406435663,"62":-0.19838037070691125,"63":-0.3001925963925029,"64":-0.23256764448826606,"65":-0.14799296766318337,"66":-0.3349986623112963,"67":0.10127371224734884,"68":0.6962841696304535,"69":-0.145467850902497,"70":0.44787323035395143,"71":0.1710679394305947,"72":0.2531323799238796,"73":-0.033926404218526184,"74":0.3931384301323091,"75":0.10493876731256122,"76":0.20747980989072867,"77":-0.15607252104111388,"78":0.373161977681637,"79":0.5712779471298723,"80":0.2905226926225534,"81":-0.0930320774641129,"82":-0.2228937276350458,"83":-0.2912465520148513,"84":-0.12001553125397726,"85":0.14739733887437223,"86":0.024080826077294983,"87":-0.048949655655413,"88":0.3831857699874768,"89":0.42296635556118467,"90":-0.11151472059044536,"91":-0.024540690552889134,"92":-0.08552746031971178,"93":-0.01016485325070729,"94":-0.005438644159679074,"95":-0.07970918264536425,"96":-0.08869575399201877,"97":0.06248352252156122,"98":0.15095232223113544,"99":0.015125793416784357}},"Wfh1":{"n":10,"d":10,"w":{"0":-0.4552102589519788,"1":0.4871163506178602,"2":-0.5178061526799773,"3":0.8563841935100368,"4":-0.5807567383087466,"5":-0.5587220264613614,"6":0.010900470143817464,"7":-0.4917409615386014,"8":-0.5313521637675858,"9":0.576500314581463,"10":-0.38419382006267955,"11":0.47819405261989645,"12":-0.25978421148988307,"13":0.8608048272390568,"14":-0.4450245434505944,"15":-0.43731189224246114,"16":-0.0675732300165116,"17":-0.42037201466013935,"18":-0.42982731572997107,"19":0.3122638716284124,"20":0.2564389588670115,"21":-0.3855619165017787,"22":0.328818851815004,"23":0.46491017100011556,"24":0.36763796378306907,"25":0.25806906493376286,"26":0.04232711703801251,"27":0.34426098980422754,"28":0.4024693248456909,"29":-0.251356768515951,"30":0.4524561920070508,"31":-0.4515979321291055,"32":0.3773367582410378,"33":0.19015459734064297,"34":0.5086763106696731,"35":0.43967375682035137,"36":-0.13351394766116909,"37":0.44063238204351957,"38":0.36597365941967186,"39":-0.44991256966486637,"40":-0.41115595333316807,"41":0.4208019763010959,"42":-0.3545541819219274,"43":0.82981312131377,"44":-0.3758857648551546,"45":-0.34131561804789223,"46":-0.005613023940929665,"47":-0.4564044909830154,"48":-0.3410291359417759,"49":0.3280409758206177,"50":-0.47779600249649634,"51":0.5613165222752279,"52":-0.35354879516253807,"53":0.9478386047605145,"54":-0.5340088984512645,"55":-0.5413423981190512,"56":0.037918900226828305,"57":-0.5013174910134004,"58":-0.4562712226702159,"59":0.5361055298618829,"60":0.5993701934989193,"61":-0.4356930200069142,"62":0.46886479160020006,"63":0.03460866264943846,"64":0.463692642132788,"65":0.5405868930878894,"66":-0.07081838011902208,"67":0.3687631726113781,"68":0.47334120643196564,"69":-0.5881702018241757,"70":-0.40011319903593534,"71":0.460045191319504,"72":-0.16361258329166642,"73":0.7755897023872547,"74":-0.46519496835868024,"75":-0.34818853564184044,"76":0.08381266650580078,"77":-0.2623606207995878,"78":-0.39579927887479394,"79":0.43557159189200684,"80":-0.12949615620783067,"81":0.1449764491920174,"82":0.06099674167696604,"83":0.775776812178531,"84":-0.06095429910998349,"85":-0.14487034536647392,"86":-0.036644312922938405,"87":-0.10553801927515168,"88":-0.02306301614883974,"89":0.20911045958286129,"90":-0.5320876149421249,"91":0.528282583473315,"92":-0.5265862495138932,"93":0.886509019058255,"94":-0.436976478515679,"95":-0.5228411148369958,"96":0.020720311089269007,"97":-0.5275633586548975,"98":-0.5528486701765541,"99":0.48872806025063814}},"bf1":{"n":10,"d":1,"w":{"0":0.5858908782530794,"1":0.5136236812757408,"2":-0.08871203020863365,"3":-0.44944221500279835,"4":0.536980610506149,"5":0.542954876095599,"6":-0.7305894390035078,"7":0.6544451681831432,"8":0.4119828020301949,"9":0.578412617921022}},"Wox1":{"n":10,"d":10,"w":{"0":0.2444105782382886,"1":0.275998314503612,"2":-0.013031327979419294,"3":-0.5815957369569348,"4":0.3073361311177228,"5":0.13377717163939204,"6":0.5317436621286353,"7":0.11991050023259263,"8":0.8126778140394539,"9":0.032519188140459274,"10":0.04139747205951887,"11":0.2823574796317763,"12":-0.16949809227632529,"13":-0.5797519427252681,"14":0.30971181982168594,"15":0.19642083980581992,"16":0.5908059005555981,"17":0.15236388608515275,"18":1.014011675544754,"19":-0.020828978168666367,"20":-0.15308328076783168,"21":-0.09650949621849608,"22":-0.15884691864638018,"23":-0.1530406755968457,"24":-0.024257049799879034,"25":-0.10609148938487244,"26":-0.07012514230294202,"27":0.06137228555207684,"28":0.793335138492933,"29":-0.2834560955532966,"30":-0.16691654620474017,"31":-0.35844682318771004,"32":-0.5724653542121069,"33":-0.6102658531178659,"34":-0.17787561753309356,"35":-0.44267975529441445,"36":0.023703144107053434,"37":0.6080136103897343,"38":-0.41161786838663317,"39":-0.16410472070257895,"40":0.23078648883588274,"41":0.6026524895535867,"42":0.11144839463650427,"43":-0.2930334349148676,"44":0.5364939533361122,"45":0.18078523627223853,"46":0.507232355345284,"47":0.09186543826299617,"48":0.6285101346016961,"49":-0.031703630952031506,"50":0.7437589133508757,"51":0.13106924774207293,"52":-0.7657540158774656,"53":-1.0554155601535518,"54":-0.1405076809368466,"55":0.2252394031496409,"56":0.8155257290643533,"57":0.4772390227623834,"58":1.3963899458486626,"59":0.45158088613210706,"60":-0.2494579412076,"61":-0.26982302144226905,"62":-0.24265427430718797,"63":-0.27850171053442424,"64":-0.31866097702340657,"65":-0.19859198420812582,"66":-0.21820732992598413,"67":0.21537279906266155,"68":0.93797623535295,"69":0.17998763879268007,"70":1.0970666014230246,"71":0.06631105313196689,"72":-0.8479025123714022,"73":-1.2927044970597292,"74":-0.15630786567561322,"75":0.47475082052000844,"76":1.0371405352285585,"77":0.48633612137303345,"78":1.6768830445021103,"79":0.8829221232679735,"80":0.5734769932791258,"81":-0.03996093878819595,"82":-0.6717214592197593,"83":-0.9357385233048537,"84":-0.043908573958431535,"85":0.27478640908820673,"86":0.9484131791364586,"87":0.44977522806510506,"88":1.3598734731832993,"89":0.5176282164979796,"90":-0.005760687511640914,"91":0.4854345396698068,"92":-0.023634523819765746,"93":-0.3955228821755939,"94":0.37580861921998837,"95":0.1388566716546376,"96":0.26737024269115156,"97":0.0807879686158595,"98":0.5551564577265375,"99":0.07301651960844838}},"Woh1":{"n":10,"d":10,"w":{"0":0.15156826074577773,"1":0.027231554235293408,"2":0.17557273786217859,"3":0.5842317460221534,"4":0.06866626208290227,"5":-0.014080073693577392,"6":-0.368336127719821,"7":-0.039531347119626636,"8":-0.06951553200860383,"9":-0.17287563338445175,"10":0.30239669148183834,"11":-0.13284724296621953,"12":0.21866613348853778,"13":0.5724729823311892,"14":0.3648659242999811,"15":0.20550060650391788,"16":-0.6828158626465003,"17":0.08807484560654519,"18":0.19568593896050304,"19":-0.28240735607842105,"20":0.30638615225620913,"21":-0.29737130788921745,"22":0.3083179643892761,"23":0.5800877695660761,"24":0.3628379136041284,"25":0.33933969076626685,"26":-0.26751671101183566,"27":0.34530180324002824,"28":0.3902505745392339,"29":-0.38258922099780335,"30":0.3610425922466067,"31":-0.38226497756977257,"32":0.31345269586776786,"33":-0.10768169849834902,"34":0.2614294523126732,"35":0.3386564007686446,"36":0.03854243511350809,"37":0.4395352882272123,"38":0.34770332008354693,"39":-0.3240123707008674,"40":0.041486873960228786,"41":-0.10699186730637225,"42":0.046338127885678725,"43":0.6491163060592019,"44":0.06579797716767742,"45":-0.01163178483480901,"46":-0.4338308947793173,"47":-0.12325962199881232,"48":-0.12397021122062037,"49":-0.07592870260552158,"50":0.28436503474262814,"51":-0.3064779827742021,"52":0.20889570832181525,"53":0.7944471516614335,"54":0.27430626307941136,"55":0.19059857635448912,"56":-0.29102291147563697,"57":0.22536018460395194,"58":0.29001644937846827,"59":-0.297274121569921,"60":0.5859463219875082,"61":-0.4780436822309947,"62":0.5105622910239871,"63":0.27639228408583394,"64":0.441853273106134,"65":0.607218090394046,"66":0.08926434962960185,"67":0.5966096770846161,"68":0.482016752384031,"69":-0.5904826854262758,"70":0.3614792554890245,"71":-0.2967619321662854,"72":0.3374149843397189,"73":0.6684808545805953,"74":0.39271910974810875,"75":0.19443150442710053,"76":-0.3654461536593978,"77":0.21167589893755806,"78":0.3113341328840965,"79":-0.2765525806348479,"80":0.3554773681496811,"81":-0.3665439563192948,"82":0.3340442233684809,"83":0.6370750616206704,"84":0.3087900683023522,"85":0.3373469803658815,"86":-0.4682695134085746,"87":0.2949219843188024,"88":0.32331440398829536,"89":-0.40416581544445634,"90":0.07942475571205641,"91":-0.07669934452385495,"92":0.11673272486378292,"93":0.5458998435407368,"94":0.16280774822511113,"95":-0.06288090734682425,"96":-0.31850398252638823,"97":0.024232029571796113,"98":0.022780947664683852,"99":-0.16834142303648086}},"bo1":{"n":10,"d":1,"w":{"0":0.019204564773500174,"1":-0.1928551468550114,"2":-0.3208158524263836,"3":-0.23634884279550222,"4":0.011072766158833014,"5":-0.1727802879598247,"6":-0.4620299298654807,"7":-0.21029226643154794,"8":-0.20738982523183053,"9":-0.035295540530265915}},"Wcx1":{"n":10,"d":10,"w":{"0":-0.06472863784540342,"1":-0.0897700338206694,"2":-0.00014605303906571566,"3":-0.09973631909535952,"4":-0.09219000780153856,"5":0.06272519129076921,"6":-0.03476974832931886,"7":-0.02018635815867354,"8":-0.027784623664481922,"9":-0.020120615974993337,"10":-0.051445953100609676,"11":-0.035977511805882864,"12":-0.020171697939598825,"13":-0.07133018026952788,"14":0.033300748743435755,"15":0.008608889578733632,"16":0.07018023369143729,"17":0.004682395936750345,"18":0.08431148067490613,"19":-0.06671880672510232,"20":0.16678608412401205,"21":0.14285819961250365,"22":0.02376111145377774,"23":0.12723894231855462,"24":0.10534501736465363,"25":0.026932156977236762,"26":0.22562656830163752,"27":0.03005235176187584,"28":-0.029260607059144215,"29":0.24484493664071938,"30":-0.38706932005444317,"31":-0.3496091252492525,"32":-0.3803036415674029,"33":-0.49736134668162846,"34":-0.07692680153406693,"35":-0.4115680461226572,"36":-0.24072913988764363,"37":0.4479457392765287,"38":-1.0684923536836852,"39":-0.8654378798114285,"40":0.007761477049264142,"41":0.003330566145453879,"42":-0.13239814082405882,"43":-0.018819829668619763,"44":-0.06014106579016606,"45":0.031549704769633875,"46":0.02938665890281844,"47":0.04904444777583095,"48":0.06313081626511749,"49":0.016617291012079878,"50":0.1929169210229484,"51":-0.03689528613832397,"52":-0.06760164734333914,"53":0.08293780733842304,"54":-0.024136021604477528,"55":-0.009264708166394731,"56":0.0934943271395479,"57":0.0100189893691286,"58":0.04464491290503261,"59":0.1033855488269867,"60":-0.1568943762169396,"61":-0.3026185573908742,"62":-0.15882225969559677,"63":-0.20417753658846663,"64":-0.16792891652297734,"65":-0.253256596366195,"66":-0.26383417846365953,"67":0.20394009489706627,"68":0.10144603229031987,"69":-0.27995440783337266,"70":0.16312188242241515,"71":0.08618966566248619,"72":-0.05904770990209771,"73":-0.0040506018420928686,"74":0.08796221345098673,"75":-0.014525148692205865,"76":0.05865174858929942,"77":0.007618346209088462,"78":-0.08501676498943832,"79":0.08200309204740511,"80":0.012101092610742346,"81":0.05897482525782019,"82":0.003984763258464193,"83":-0.01109947063728621,"84":0.07867299643963722,"85":-0.00677234548659734,"86":0.08199031366627718,"87":-0.006601003283165683,"88":-0.05564560292826893,"89":0.010043773639317185,"90":0.08061114071367201,"91":0.034191484885530324,"92":0.03792577511965204,"93":0.11157189460739364,"94":0.12149167375217283,"95":0.02890434587301099,"96":0.09333320506538588,"97":-0.0339638566974719,"98":0.023120455626805144,"99":0.03004688581894983}},"Wch1":{"n":10,"d":10,"w":{"0":0.6520194709960113,"1":-0.5090676794897526,"2":0.5263446101222016,"3":-0.6936860697175659,"4":0.5827472434366432,"5":0.6636988718561195,"6":-0.057698532276461524,"7":0.5858264716400489,"8":0.6452476739422282,"9":-0.662451308509659,"10":-0.5472265111873983,"11":0.5552098337032342,"12":-0.6348164846357461,"13":0.7375014558714088,"14":-0.6400619256140638,"15":-0.6686071929469051,"16":-0.010677455508058206,"17":-0.5697732170634876,"18":-0.5343763548892942,"19":0.5671040205779568,"20":0.44320357039498837,"21":-0.37245664183555227,"22":0.3666875998758891,"23":-0.8570250583629228,"24":0.405371567455289,"25":0.45065497201989646,"26":-0.051639529222827135,"27":0.4335507930836567,"28":0.4708157865870026,"29":-0.4468086654228988,"30":-0.049947414371323495,"31":-0.02889353427006262,"32":-0.02296266341217226,"33":-0.5759267086561453,"34":-0.02416773634741658,"35":-0.08101381381687052,"36":-0.8783950082346521,"37":0.05735119167405014,"38":-0.03418012128559726,"39":0.07399333957515167,"40":0.5885631674699449,"41":-0.6746583535990835,"42":0.5947964296562845,"43":-0.6188470104023003,"44":0.6242415494326284,"45":0.5548619808154378,"46":0.025540123636159533,"47":0.635904968380953,"48":0.6134287396416238,"49":-0.5991111750937427,"50":0.6642772640027212,"51":-0.5879383462778703,"52":0.5396655559120853,"53":-0.7870151102180751,"54":0.6746435982112269,"55":0.5931455015259016,"56":0.017825188304832297,"57":0.5014945602228218,"58":0.5256249339776919,"59":-0.6602307712620678,"60":0.36584844284498275,"61":-0.3707608914695049,"62":0.3911550382482313,"63":0.49743880672094776,"64":0.4594506916814957,"65":0.4306682989208451,"66":-0.6547332242330233,"67":0.4005779641657855,"68":0.417179412983662,"69":-0.48725063689389764,"70":0.511875054415766,"71":-0.6018878442884253,"72":0.6210916380437262,"73":-0.8457813537528198,"74":0.5363456648948057,"75":0.4933491343152506,"76":-0.035651328485476426,"77":0.5197613375590422,"78":0.6287426739638605,"79":-0.5473399874912078,"80":0.48228486965787465,"81":-0.5543166308155416,"82":0.5578117053748198,"83":-0.8599541974722006,"84":0.6340713901987419,"85":0.6338570807252601,"86":0.03969216147182824,"87":0.574433646161699,"88":0.6424233580591564,"89":-0.6248525869170839,"90":-0.6926015200394152,"91":0.5400171860186624,"92":-0.6061414123459682,"93":0.6626710110620374,"94":-0.6317532438468771,"95":-0.5989839189651249,"96":0.07775917879576451,"97":-0.4994959633823388,"98":-0.527274793265939,"99":0.6772523283050221}},"bc1":{"n":10,"d":1,"w":{"0":-0.9285661262112621,"1":0.8025508561765364,"2":-0.7750764818364341,"3":0.5404688655209992,"4":-0.7605504611298554,"5":-0.8624541221283022,"6":0.18853460458992516,"7":-0.7354213564130023,"8":-0.9151201381983766,"9":0.8743018268832787}},"Whd":{"n":51,"d":10,"w":{"0":-0.01446671912358085,"1":0.23191388338026026,"2":-0.26052149636783084,"3":-0.8447628612779611,"4":0.0651864469094249,"5":-0.3686359571655619,"6":0.04892581535350611,"7":-0.49057675665371947,"8":-0.43310011847482355,"9":-0.0017587271727069372,"10":0.9901160787315383,"11":-0.840188347403209,"12":0.44291348254668195,"13":-0.012976626507823795,"14":1.0466488933391769,"15":0.7946561338378972,"16":-0.30622072901555847,"17":0.5804799563292482,"18":0.6884673080827207,"19":-0.9340900323397064,"20":0.8806080019405961,"21":-0.8097058583086176,"22":0.37120287992987666,"23":0.05998865763826686,"24":0.944869624909582,"25":0.7910381892682737,"26":-0.33121684816730707,"27":0.6176196223947397,"28":0.5997668182050966,"29":-0.8721830087767931,"30":1.4695256409876312,"31":-1.2203728517700896,"32":0.7418585968236017,"33":-0.28897914313422035,"34":1.4825709564249248,"35":1.1905577753406638,"36":-0.3218081120575588,"37":0.9418374400848398,"38":1.0458523870048417,"39":-1.291828464747836,"40":1.4207757160188301,"41":-1.2017027427848332,"42":0.6361448537153507,"43":-0.2925459686106377,"44":1.336066907833075,"45":1.1492008538797194,"46":-0.2817371617455154,"47":0.8979420692111536,"48":1.0680191519283189,"49":-1.2551771135253356,"50":1.349361903350904,"51":-1.2345171374108224,"52":0.6780448977370105,"53":-0.24321481574084727,"54":1.4293950097669543,"55":1.1552791441397934,"56":-0.32960838476521787,"57":0.9382355879503999,"58":1.1191057838825902,"59":-1.2857313447488607,"60":1.1434773735115173,"61":-0.9382358264570265,"62":0.5811678518570785,"63":-0.046866316699825605,"64":1.038217939499126,"65":0.8689548107507532,"66":-0.35254328425681264,"67":0.7236512652838354,"68":0.9009977153501102,"69":-1.0443242141042621,"70":1.3815751976906157,"71":-1.0921804721983392,"72":0.6028751930796191,"73":-0.2589187686253344,"74":1.4569859919012025,"75":1.1841067609422633,"76":-0.23343537793886526,"77":0.8980621829973298,"78":1.0600758346402295,"79":-1.3604537792498372,"80":1.4798628009993349,"81":-1.1409572651631743,"82":0.6356008446239673,"83":-0.2259024621241123,"84":1.4387049954889881,"85":1.0725618083113713,"86":-0.34089201659906015,"87":0.918679966795837,"88":1.0173297658948188,"89":-1.2627165743593634,"90":1.481574469748417,"91":-1.2055959261074969,"92":0.6977531559196752,"93":-0.2824857117159876,"94":1.3794543804913015,"95":1.1143034135298082,"96":-0.22876790384036041,"97":0.9974716502636681,"98":1.1190670045577615,"99":-1.2876442074692231,"100":1.1576595460449663,"101":-0.8596290350324961,"102":0.5143020541194028,"103":-0.04694723228475678,"104":1.1520474258183693,"105":0.8265319453690418,"106":-0.30408142996454396,"107":0.6114022794310708,"108":0.8348891515949042,"109":-0.9787419794246208,"110":-0.29378928057200815,"111":0.18288121540444058,"112":-0.12083459518581877,"113":0.2842338298822385,"114":-0.23578482190793687,"115":-0.006589535455651593,"116":0.2796395188927166,"117":0.11400042232157939,"118":-0.0008026322239220243,"119":0.32136025908406846,"120":-0.049151258891184685,"121":0.14691650207434137,"122":-0.12702513366309928,"123":-1.4589100133841852,"124":0.016953067175633683,"125":-0.2824460775982687,"126":-0.4433278408981403,"127":-0.37542328965613136,"128":-0.3499688788291107,"129":0.08887841644874994,"130":-0.39656274778177486,"131":0.46315977574734984,"132":-0.5764753221129854,"133":0.39670985141188636,"134":-0.34911674691019945,"135":-0.35074451596294315,"136":0.915560692040627,"137":-0.44704302356966813,"138":-0.3264487509287659,"139":0.44089160085704443,"140":-0.6027652196545856,"141":0.4766329317628525,"142":-0.42610961831579947,"143":0.061876853400090595,"144":-0.5746413990808465,"145":-0.7525281590407561,"146":0.3211371798100027,"147":-0.8095596166144056,"148":-0.5626470917805216,"149":0.5514126663508966,"150":0.17737084173417575,"151":-0.04136975796869849,"152":0.21999994876595727,"153":-0.013448615028361241,"154":0.09185926532011802,"155":0.23687841685914804,"156":-0.23328672747077345,"157":0.2874111725220099,"158":0.13273969756861698,"159":-0.11390594888040724,"160":-0.24586927641817122,"161":0.182959426187048,"162":-0.1689442390053859,"163":0.19077342876559372,"164":-0.13457098070018034,"165":-0.2108065163173806,"166":0.10731251271249931,"167":-0.2521961838923543,"168":-0.22271131060277963,"169":0.21601780883856822,"170":0.13115864233009486,"171":-0.17636273334287947,"172":0.11319272237718486,"173":0.07619881758447891,"174":0.20150189026432208,"175":0.32168810206805026,"176":0.7911713444631203,"177":0.18977600633623354,"178":0.24053313031307164,"179":-0.017067614187185026,"180":0.21484846608561392,"181":-0.0964991733785573,"182":0.27271201253001814,"183":-0.4800507322566262,"184":0.07923239565732339,"185":0.21046076722574475,"186":-0.2974190980105307,"187":0.2071707961133397,"188":0.20625038436729642,"189":-0.1345542271771111,"190":-0.23592468571758426,"191":0.21301625966698523,"192":-0.15246983960256358,"193":-0.009251564790491329,"194":-0.2507893667459771,"195":-0.22188818998231413,"196":0.1447446182075861,"197":-0.09061486911366028,"198":-0.2115487316893176,"199":0.27156338615396136,"200":-0.1414703877445569,"201":0.12233467954442044,"202":-0.003225892722902349,"203":0.2720008432733508,"204":-0.18570261581397451,"205":0.10051606120034434,"206":0.11791532029347801,"207":0.14944906014523057,"208":0.09086438093083002,"209":0.18144861925749983,"210":-0.020025559563139707,"211":0.10984974035509483,"212":-0.09956296520802392,"213":0.12081156593650608,"214":-0.031254737171202945,"215":-0.07673882267687783,"216":0.25023295286562297,"217":-0.06330273068010601,"218":-0.049546784315796816,"219":0.16128459549065166,"220":-0.16839938228973766,"221":0.11108572077228837,"222":-0.09350970376228136,"223":-0.08229863241253572,"224":-0.19339265102236858,"225":-0.19644970095430436,"226":-0.2709645336671647,"227":-0.003993080661457808,"228":-0.034977637440358304,"229":0.2263032899555408,"230":0.33480213914959234,"231":-0.33056755070572164,"232":0.5007892498984529,"233":-0.48430809585783763,"234":0.35171947237909307,"235":0.13551687240364782,"236":-0.22724671563788293,"237":0.10393186823690458,"238":0.15692546829168932,"239":-0.4624616826588699,"240":0.28077498876835316,"241":-0.30195463699034225,"242":0.19845627675228525,"243":-0.08614477132585538,"244":0.1998399083267277,"245":0.43055002856532243,"246":0.3498352912258324,"247":0.5675266106724677,"248":0.4293816908389519,"249":-0.2604490242737785,"250":-0.16200497459009974,"251":0.1324162220235675,"252":0.05662597465857694,"253":-0.250498739920937,"254":-0.022045884640637115,"255":-0.10273380803261878,"256":-0.07582635756950212,"257":-0.07350013144732637,"258":0.008958367147584764,"259":0.12829079348853936,"260":-0.021447466027655408,"261":-0.040354545574707135,"262":0.040965105321125704,"263":-0.15445365983501513,"264":-0.06212437209061869,"265":0.21106186437491128,"266":0.2378773456609931,"267":0.2713259805593438,"268":0.21634693376125025,"269":0.004277671680815873,"270":0.1197787313850274,"271":-0.14140889833287051,"272":0.04410961995768975,"273":-0.031385593213052815,"274":0.1916398053538514,"275":0.16401435319092617,"276":-0.06474303285805866,"277":0.2495562302225974,"278":0.2377927100704042,"279":-0.17019487390576102,"280":-0.3270547983639188,"281":0.26822755459091197,"282":-0.09036114988759915,"283":0.042269615692325915,"284":-0.27622552387876004,"285":-0.21682922941217947,"286":0.2597104295340946,"287":-0.27209507199383115,"288":-0.30850841157358866,"289":0.2169664162508394,"290":0.00937798114826531,"291":-0.16296114145741536,"292":0.027657546347885014,"293":0.06495868147016556,"294":0.059814915922725836,"295":0.15888330205046408,"296":0.2091732961798643,"297":0.2530042389069185,"298":0.11549408326359628,"299":-0.039229323674931434,"300":0.08264412826830267,"301":-0.05948133552359461,"302":-0.09537399033504973,"303":0.7978570870745937,"304":0.057863404701063416,"305":-0.07609215319952119,"306":-0.5241806919062855,"307":-0.10669813720003365,"308":-0.016524673247053322,"309":-0.12951617078546418,"310":0.08756116188603871,"311":-0.0994680116079669,"312":0.10221186147607376,"313":-0.6547009022892238,"314":0.0889329181437847,"315":0.03372561401201979,"316":-0.24220033423452744,"317":-0.08057091790020993,"318":0.07741428886977732,"319":-0.08941846162833009,"320":1.3319235593616645,"321":-1.1932567911386442,"322":0.7043703797770232,"323":-0.29718638915048007,"324":1.3893719866102927,"325":1.093865740805803,"326":-0.2265375316661198,"327":0.9163854320410971,"328":1.0005133109527293,"329":-1.336066129242873,"330":0.18422409504726572,"331":-0.04460106991518073,"332":-0.13454718891757048,"333":-0.9584579080167956,"334":0.17471850416389012,"335":0.07753907223358406,"336":-0.282315343639481,"337":-0.045433036439005134,"338":0.00749937691034518,"339":-0.06183198010729862,"340":-0.20497086129945466,"341":0.1455898998099373,"342":0.029732996699778726,"343":0.3492481493658408,"344":-0.23167756214440494,"345":-0.06957504423290503,"346":0.15359369899660053,"347":-0.12418075500730916,"348":-0.07469261274938908,"349":0.2082274464199613,"350":-0.05879023372027535,"351":-0.10736489325059176,"352":0.02348189132995563,"353":-0.45316140904649355,"354":-0.14964935923711442,"355":0.19887750504744733,"356":0.15870200154954553,"357":0.3748402106850415,"358":0.3055846289983747,"359":0.14077270883452764,"360":1.3862495866961904,"361":-1.1710573873346017,"362":0.5928608383060049,"363":-0.20162765368130478,"364":1.397762445746749,"365":1.0499486169063634,"366":-0.21945308754371626,"367":0.9299222838557809,"368":1.0980134300141,"369":-1.3588438060862915,"370":1.3929956147851732,"371":-1.2298212112368738,"372":0.7150020743531228,"373":-0.212585013841618,"374":1.3778639456540525,"375":1.079568626764271,"376":-0.23795876760397555,"377":0.9588754723139585,"378":1.06298405183011,"379":-1.2235777240553627,"380":0.7721873906052809,"381":-0.6134121397634067,"382":0.6120140103782402,"383":0.02422061211885669,"384":0.6532960945982005,"385":0.6828083897160295,"386":-0.30209134053394815,"387":0.6736077535124365,"388":0.7603404778111853,"389":-0.5960905543758227,"390":1.3353436339270859,"391":-1.2564223249578808,"392":0.660196015445483,"393":-0.31252556067212944,"394":1.345062591022567,"395":1.1615143701285413,"396":-0.25084609113982376,"397":0.8679624860411712,"398":1.022975317718009,"399":-1.353632078359136,"400":1.420102204386804,"401":-1.1322159237501346,"402":0.6120057701886632,"403":-0.32131442763222784,"404":1.45904004914641,"405":1.1568776965124248,"406":-0.33656816486006436,"407":0.8929004217535207,"408":1.02872260966131,"409":-1.2561879135037695,"410":1.4146615617392384,"411":-1.108132011428863,"412":0.7228976996219569,"413":-0.2658692199267319,"414":1.3781351904023316,"415":1.0757602885984743,"416":-0.337643997856838,"417":0.9012580630540679,"418":1.077309548548881,"419":-1.2773150371011044,"420":1.1395951867507952,"421":-0.9530589420395243,"422":0.5949366252587717,"423":-0.23876324967346357,"424":1.1638548721915825,"425":0.9970041053774077,"426":-0.33539376876327315,"427":0.8358915162510085,"428":0.8388093153534991,"429":-0.9648808891894087,"430":1.483115920581194,"431":-1.1558023145365353,"432":0.7330737319245881,"433":-0.2869382731194322,"434":1.4312922708318578,"435":1.1981972238441672,"436":-0.3281268163420065,"437":1.0003644571765071,"438":1.0700345322622313,"439":-1.2807828319435486,"440":1.052356127084387,"441":-0.9514900219849144,"442":0.6205062589298587,"443":-0.32006773780223474,"444":1.0440574321232312,"445":1.0020888612236687,"446":-0.27447328803105325,"447":0.8780475585111558,"448":0.871370486731125,"449":-1.020873837995315,"450":1.4287783737047888,"451":-1.10613841132323,"452":0.5958598448973975,"453":-0.19251496713439223,"454":1.3916582917228437,"455":1.1110742730748022,"456":-0.28288901680541434,"457":0.9220784330122229,"458":1.010156338830542,"459":-1.3640238970379701,"460":1.4186888355904443,"461":-1.1837401199441888,"462":0.6288903754198205,"463":-0.20517224262240158,"464":1.4841291664055738,"465":1.1031126021688775,"466":-0.3324244349949003,"467":0.9902772044202303,"468":1.0064385752796021,"469":-1.2691020884066146,"470":1.161344868816394,"471":-0.9298755027163801,"472":0.5674202337384937,"473":-0.2983589111538273,"474":1.1065675077363177,"475":1.0207121812642213,"476":-0.301918872151383,"477":0.8601908602833749,"478":0.9023846711958036,"479":-1.13774910472222,"480":1.3810325240099026,"481":-1.213281125927754,"482":0.6523850362860241,"483":-0.20446033577194317,"484":1.470620989778134,"485":1.1888024883100006,"486":-0.2628725922867086,"487":0.9753740836838254,"488":1.070893896718536,"489":-1.2861697129253316,"490":1.3734201573353169,"491":-1.2068584406168672,"492":0.6011842527600605,"493":-0.22994226144607854,"494":1.4383004575265008,"495":1.0964647181746463,"496":-0.2650236959285496,"497":0.9812350807607748,"498":1.0340207579097689,"499":-1.2602838030488102,"500":1.3361395672904417,"501":-1.1946617404389703,"502":0.7334601825471553,"503":-0.3250487180883795,"504":1.4580612542785454,"505":1.1997259875500064,"506":-0.2605910267707237,"507":0.8562938373635012,"508":0.9758389300349887,"509":-1.3716602640813302}},"bd":{"n":51,"d":1,"w":{"0":-0.5446264578915779,"1":-0.6059434778758691,"2":-0.6161083461715089,"3":-0.8473700424756562,"4":-0.8759209786033737,"5":-0.864574182674815,"6":-0.7207218821157092,"7":-0.8626858785706905,"8":-0.8691967299457708,"9":-0.8565348838514554,"10":-0.6193470053227423,"11":0.9737700270929053,"12":-0.024555173327014318,"13":0.21047915813131976,"14":0.1580232875216196,"15":0.3611241156967567,"16":0.368965551860584,"17":0.16160324676226906,"18":0.2489273301474313,"19":0.5756785033040214,"20":0.7607224671329931,"21":-0.06375313959261447,"22":0.6707224612123488,"23":-0.6109260418911525,"24":0.46148709298042556,"25":0.6713487271656376,"26":0.5892990280734414,"27":0.05971204116590223,"28":0.6484458794464694,"29":0.04259037838300319,"30":0.7555737343880716,"31":-0.5732368951391306,"32":-0.8762986839413108,"33":-0.6253120436211176,"34":0.014791326371372607,"35":0.25291882547017996,"36":-0.8681457096142418,"37":-0.8764958450793677,"38":-0.19014668351409839,"39":-0.8725364172357487,"40":-0.8696099723600326,"41":-0.8797588541057327,"42":-0.7672321525950593,"43":-0.8531073015592116,"44":-0.8053953141083502,"45":-0.8699900017167599,"46":-0.8633405914813941,"47":-0.7664915813449282,"48":-0.8569734525039205,"49":-0.8708050978202505,"50":-0.8626921644667596}}},"letterToIndex":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9,"9":10,"t":11,"h":12,"e":13," ":14,"c":15,"o":16,"m":17,"p":18,"a":19,"n":20,"y":21,"s":22,",":23,"f":24,"r":25,"u":26,"w":27,"i":28,"b":29,"l":30,"'":31,"x":32,"v":33,"d":34,"g":35,":":36,";":37,"k":38,"\"":39,"-":40,"z":41,"j":42,"?":43,"q":44,"[":45,"]":46,"$":47,"(":48,")":49,"%":50},"indexToLetter":{"1":"0","2":"1","3":"2","4":"3","5":"4","6":"5","7":"6","8":"7","9":"8","10":"9","11":"t","12":"h","13":"e","14":" ","15":"c","16":"o","17":"m","18":"p","19":"a","20":"n","21":"y","22":"s","23":",","24":"f","25":"r","26":"u","27":"w","28":"i","29":"b","30":"l","31":"'","32":"x","33":"v","34":"d","35":"g","36":":","37":";","38":"k","39":"\"","40":"-","41":"z","42":"j","43":"?","44":"q","45":"[","46":"]","47":"$","48":"(","49":")","50":"%"},"vocab":["0","1","2","3","4","5","6","7","8","9","t","h","e"," ","c","o","m","p","a","n","y","s",",","f","r","u","w","i","b","l","'","x","v","d","g",":",";","k","\"","-","z","j","?","q","[","]","$","(",")","%"]}

In [5]:
karpathy_js['model'].keys()


['bd',
 'Wfx1',
 'Wfx0',
 'bi0',
 'bi1',
 'Whd',
 'Wox0',
 'Wox1',
 'bo0',
 'bo1',
 'Wcx0',
 'Wcx1',
 'Wfh1',
 'Wfh0',
 'bc0',
 'bc1',
 'Wix0',
 'Wix1',
 'Wil',
 'Wih0',
 'Wih1',
 'bf1',
 'bf0',
 'Woh0',
 'Woh1',
 'Wch0',
 'Wch1']

In [99]:
karpathy_js.keys()

['vocab',
 'generator',
 'letter_size',
 'hidden_sizes',
 'indexToLetter',
 'letterToIndex',
 'model']

In [121]:
[k for k in karpathy_js['model'].keys() if '0' in k]

['Wfx0',
 'bi0',
 'Wox0',
 'bo0',
 'Wcx0',
 'Wfh0',
 'bc0',
 'Wix0',
 'Wih0',
 'bf0',
 'Woh0',
 'Wch0']

In [123]:
karpathy_js['model'].keys()

['bd',
 'Wfx1',
 'Wfx0',
 'bi0',
 'bi1',
 'Whd',
 'Wox0',
 'Wox1',
 'bo0',
 'bo1',
 'Wcx0',
 'Wcx1',
 'Wfh1',
 'Wfh0',
 'bc0',
 'bc1',
 'Wix0',
 'Wix1',
 'Wil',
 'Wih0',
 'Wih1',
 'bf1',
 'bf0',
 'Woh0',
 'Woh1',
 'Wch0',
 'Wch1']

In [130]:
# karpathy_js['model']['bd'] # 1x51
# karpathy_js['model']['Whd'] # 10x51
# karpathy_js['model']['Wil'] # 5x51
[k for k in karpathy_js['model'].keys() if '0' not in k and '1' not in k]

['bd', 'Whd', 'Wil']

In [27]:
'Wil' in karpathy_js['model'].keys()

True

In [36]:
karpathy_js['model']['bo0']

{'d': 1,
 'n': 10,
 'w': {'0': 0.1829743361785246,
  '1': -0.297794530149536,
  '2': 0.07931779110946899,
  '3': -0.029868739735111352,
  '4': -0.18213186916801755,
  '5': -0.4554960493738334,
  '6': -0.20735442180051405,
  '7': -0.272465541809876,
  '8': 0.49128766507380456,
  '9': -0.14097134092324276}}

In [21]:
karpathy_js['model'].keys()

['bd',
 'Wfx1',
 'Wfx0',
 'bi0',
 'bi1',
 'Whd',
 'Wox0',
 'Wox1',
 'bo0',
 'bo1',
 'Wcx0',
 'Wcx1',
 'Wfh1',
 'Wfh0',
 'bc0',
 'bc1',
 'Wix0',
 'Wix1',
 'Wil',
 'Wih0',
 'Wih1',
 'bf1',
 'bf0',
 'Woh0',
 'Woh1',
 'Wch0',
 'Wch1']

In [49]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
text = re.sub(' +',' ',' '.join(text.splitlines()))


In [28]:
text = open('./trump.txt', 'rb').read().lower().decode('utf-8')
text = re.sub(' +',' ',' '.join(text.splitlines()))


In [29]:
set(text)

{u' ',
 u'!',
 u'"',
 u'$',
 u'%',
 u"'",
 u'(',
 u')',
 u',',
 u'-',
 u'.',
 u'/',
 u'0',
 u'1',
 u'2',
 u'3',
 u'4',
 u'5',
 u'6',
 u'7',
 u'8',
 u'9',
 u':',
 u';',
 u'?',
 u'@',
 u'a',
 u'b',
 u'c',
 u'd',
 u'e',
 u'f',
 u'g',
 u'h',
 u'i',
 u'j',
 u'k',
 u'l',
 u'm',
 u'n',
 u'o',
 u'p',
 u'q',
 u'r',
 u's',
 u't',
 u'u',
 u'v',
 u'w',
 u'x',
 u'y',
 u'z'}

In [104]:
text.decode('utf-8')

u"trump: wow. whoa. that is some group of people. thousands.\n\nso nice, thank you very much. that's really nice. thank you. it's great to be at trump tower. it's great to be in a wonderful city, new york. and it's an honor to have everybody here. this is beyond anybody's expectations. there's been no crowd like this.\n\nand, i can tell, some of the candidates, they went in. they didn't know the air conditioner didn't work. they sweated like dogs.\n\n(laughter)\n\nthey didn't know the room was too big, because they didn't have anybody there. how are they going to beat isis? i don't think it's gonna happen.\n\n(applause)\n\nour country is in serious trouble. we don't have victories anymore. we used to have victories, but we don't have them. when was the last time anybody saw us beating, let's say, china in a trade deal? they kill us. i beat china all the time. all the time.\n\n(applause)\n\naudience member: we want trump. we want trump.\n\ntrump: when did we beat japan at anything? they

In [101]:
text.encode('utf-8')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 2073: ordinal not in range(128)

In [5]:
text = open('./hillary.txt', 'rb').read().lower().decode('utf-8')
text = re.sub(' +',' ',' '.join(text.splitlines()))    

In [6]:
# tokenize the text, and do a word level rnn, since we only have 3000 words

In [9]:
import string
t = text.split(string.punctuation)

In [13]:
import nltk
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

# TO DO: Split up numbers but keep word tokenize

In [18]:
from nltk.tokenize import word_tokenize
chars = set(word_tokenize(text))
text = word_tokenize(text)

In [21]:
len(chars)

2957

In [18]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

In [19]:
len(text)

600901

In [131]:
codex = {'’': '\'', '”': '"', '—': ' - ', '…': '...', '“': ',', '–': '-', '‘':'\''}
for k, v in codex.iteritems():
    text = re.sub(k, v, text)

In [133]:
open('./trump.txt', 'wb').write(text)